<a href="https://colab.research.google.com/github/rubuntu/uaa-417-sistemas-de-gestion-de-bases-de-datos-avanzados/blob/main/pgvector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!if [ -f /etc/apt/sources.list.d/pitti-postgresql-$(lsb_release --codename --short).list ]; then rm /etc/apt/sources.list.d/pitti-postgresql-$(lsb_release --codename --short).list; fi
!echo "deb http://apt.postgresql.org/pub/repos/apt/ $(lsb_release --codename --short)-pgdg main" > /etc/apt/sources.list.d/pgdg.list
!wget --quiet -O - http://apt.postgresql.org/pub/repos/apt/ACCC4CF8.asc | apt-key add -
!apt-get update
!apt-get install pgdg-keyring

!apt-get install postgresql-17-pgvector postgresql-contrib-17

!git clone https://github.com/rubuntu/datasets.git


In [2]:
# Iniciar postgresql
!service postgresql start
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres';"
!sudo -u postgres psql -c "CREATE DATABASE ragdb;"

# Crear la extensión pgvector
!sudo -u postgres psql -d ragdb -c "CREATE EXTENSION IF NOT EXISTS vector;"
#!sudo -u postgres psql -d ragdb -c "\\dx"

 * Starting PostgreSQL 17 database server
   ...done.
ALTER ROLE
CREATE DATABASE
CREATE EXTENSION


In [3]:
%%capture
# Instalar bibliotecas necesarias
!pip install openai psycopg2-binary sentence-transformers

In [4]:
import os
import openai
import psycopg2
from psycopg2.extensions import register_adapter, AsIs
from psycopg2.extras import execute_values
from sentence_transformers import SentenceTransformer

# Configurar modelo para embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Configuración de la conexión a PostgreSQL
DB_NAME = "ragdb"
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"

def connect_to_db():
    """Establecer conexión con la base de datos PostgreSQL."""
    return psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
    )

def create_table():
    """Crear tabla para almacenar los embeddings."""
    query = """
    CREATE TABLE IF NOT EXISTS markdown_embeddings (
        id SERIAL PRIMARY KEY,
        file_path TEXT NOT NULL,
        content TEXT NOT NULL,
        embedding VECTOR(384)
    );
    """
    with connect_to_db() as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            conn.commit()

def process_markdown_files(root_dir):
    """Leer archivos Markdown y generar embeddings."""
    data_to_insert = []
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith('.md'):
                file_path = os.path.join(dirpath, filename)
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    embedding = model.encode(content).tolist()
                    data_to_insert.append((file_path, content, embedding))
    return data_to_insert

def insert_embeddings(data):
    """Insertar embeddings en la base de datos."""
    query = """
    INSERT INTO markdown_embeddings (file_path, content, embedding)
    VALUES %s
    """
    with connect_to_db() as conn:
        with conn.cursor() as cur:
            execute_values(cur, query, data, template=None)
            conn.commit()

if __name__ == "__main__":
    # Crear tabla si no existe
    create_table()

    # Procesar archivos Markdown
    root_directory = "datasets/www.indufar.com.py/"
    embeddings_data = process_markdown_files(root_directory)

    # Insertar datos en la base de datos
    insert_embeddings(embeddings_data)
    print(f"Procesados e insertados {len(embeddings_data)} archivos Markdown en la base de datos.")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Procesados e insertados 640 archivos Markdown en la base de datos.
